In [1]:
# 加载必要的库
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
# 定义超参数 (超参数是人为定义的， 参数是机器学习的)
BATCH_SIZE = 16 # 每批处理的数据多少
DEVICE = torch.device("cuda" if torch.cuda.is_available() else"cpu")# 决定是gpu还是cpu训练
EPOCHS = 10 # 训练数据集的轮次

In [3]:
# 构建pipeline，对图像做处理
pipeline = transforms.Compose([
    transforms.ToTensor(),  # 将图片转换成tensor
    transforms.Normalize((0.1307,), (0.3081,)) #正则化 降低模型的复杂度
])

In [4]:
# 下载 加载数据
from torch.utils.data import DataLoader

# 下载数据集
train_set = datasets.MNIST("data", train=True, download=True, transform=pipeline)

test_set = datasets.MNIST("data", train=False, download=True, transform=pipeline)

# 加载数据
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)

test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)

In [5]:
# import torchvision
# train_set = torchvision.datasets.MNIST(root='/Users/zhangchu/miniconda3/notebook/data/MNIST',
#                                            train=True,
#                                            transform=transforms.ToTensor(),
#                                            download=False
# )
# test_set = torchvision.datasets.MNIST(root='/Users/zhangchu/miniconda3/notebook/data/MNIST',
#                                       train=False,
#                                       transform=transforms.ToTensor(),
#                                       download=False
# )

In [6]:
# 读取图片
with open("./data/MNIST/raw/train-images-idx3-ubyte", "rb") as f:
    file = f.read()

In [7]:
image1 = [int(str(item).encode('ascii'), 16)for item in file[16 : 16+784]]
# print(image1)

In [8]:
import cv2
import numpy as np

image1_np = np.array(image1, dtype=np.uint8).reshape(28, 28, 1)

print(image1_np.shape)

(28, 28, 1)


In [9]:
cv2.imwrite("digit.jpg", image1_np)

True

In [10]:
# 构建网络模型
class Digit(nn.Module):
    def __init__(self):
        super(). __init__()
        # 定义卷积层
        self.conv1 = nn.Conv2d(1, 10, 5)# 1：灰度图片的通道， 10：输出通道， 5：kernel
        self.conv2 = nn.Conv2d(10, 20, 3)# 10：输入通道， 20：输出通道， 3：kernel
        # 定义全连接层
        self.fc1 = nn.Linear(20*10*10, 500)# 20*10*10:输入通道， 500：输出通道
        self.fc2 = nn.Linear(500, 10)#500：输入通道， 10：输出通道
        
    # 定义前向传播
    def forward(self, x):
        input_size = x.size(0)# batch_size
        x = self.conv1(x)# 输入：batch*1*28*28, 输出：batch*10*24*24 (28-5+1=24)
        x = F.relu(x)#保持shape不变，输出：batch*10*24*24  激活函数
        x = F.max_pool2d(x, 2, 2)# 输入：batch*10*24*24, 输出：batch*10*12*12
        
        x = self.conv2(x)#输入：batch*10*12*12, 输出：batch*20*10*10 (12-3+1)
        x = F.relu(x)
        
        x = x.view(input_size, -1)#拉平， -1：自动计算维度，20*10*10=2000
        
        x = self.fc1(x)# 输入：batch*2000，输出：batch*500
        x = F.relu(x)#保持shape不变
        
        x = self.fc2(x)# 输入：batch*500，输出：batch*10
        
        output = F.log_softmax(x, dim=1)# dim表示按行计算。计算分类后，每个数字的概率值
        
        return output

In [11]:
# 定义优化器
model = Digit().to(DEVICE)

optimizer = optim.Adam(model.parameters())

In [12]:
# 定义训练方法
def train_model(model, device, train_loader, optimizer, epoch):
    # 模型训练
    model.train()
    for batch_index, (data, target) in enumerate(train_loader):
        # 部署到DEVICE上去
        data, target = data.to(device), target.to(device)
        # 梯度初始化为0
        optimizer.zero_grad()
        # 训练后的结果
        output = model(data)
        # 计算损失
        loss = F.cross_entropy(output, target)
#         # 找到概率值最大的下标
#         pred = output.max(1, keepdim=True)
        # 反向传播
        loss.backward()
        # 参数优化
        optimizer.step()
        if batch_index % 3000 == 0:
            print("Train Epoch : {} \t Loss : {:.6f}". format(epoch, loss.item()))

In [13]:
# 定义测试方法
def test_model(model, device, test_loader):
    # 模型验证
    model.eval()
    # 正确率
    correct = 0.0
    # 测试损失
    test_loss = 0.0
    with torch.no_grad():     # 不会计算梯度，也不会进行反向传播
        for data, target in test_loader:
            # 部署到device上
            data, target = data.to(device), target.to(device)
            # 测试数据
            output = model(data)
            # 计算测试损失
            test_loss +=F.cross_entropy(output, target).item()
            # 找到概率值最大的下标
            pred = output.max(1, keepdim=True)[1]
            # 累计正确的值
            correct += pred.eq(target.view_as(pred)).sum().item()
        test_loss /= len(test_loader.dataset)
        print("Test --- Average loss : {:.4f}, Accuracy : {:.3f}\n".format(
              test_loss, 100.0* correct / len(test_loader.dataset)))

In [14]:
# 调用上面的方法
for epoch in range(1, EPOCHS + 1):
    train_model(model, DEVICE, train_loader, optimizer, epoch)
    test_model(model, DEVICE, test_loader)

Train Epoch : 1 	 Loss : 2.293501
Train Epoch : 1 	 Loss : 0.176619
Test --- Average loss : 0.0028, Accuracy : 98.500

Train Epoch : 2 	 Loss : 0.007827
Train Epoch : 2 	 Loss : 0.004565
Test --- Average loss : 0.0027, Accuracy : 98.480

Train Epoch : 3 	 Loss : 0.000475
Train Epoch : 3 	 Loss : 0.000021
Test --- Average loss : 0.0021, Accuracy : 98.930

Train Epoch : 4 	 Loss : 0.001975
Train Epoch : 4 	 Loss : 0.000003
Test --- Average loss : 0.0025, Accuracy : 98.780

Train Epoch : 5 	 Loss : 0.001599
Train Epoch : 5 	 Loss : 0.000382
Test --- Average loss : 0.0025, Accuracy : 98.940

Train Epoch : 6 	 Loss : 0.010042
Train Epoch : 6 	 Loss : 0.000004
Test --- Average loss : 0.0034, Accuracy : 98.730

Train Epoch : 7 	 Loss : 0.000025
Train Epoch : 7 	 Loss : 0.000000
Test --- Average loss : 0.0031, Accuracy : 98.850

Train Epoch : 8 	 Loss : 0.001381
Train Epoch : 8 	 Loss : 0.000153
Test --- Average loss : 0.0033, Accuracy : 99.020

Train Epoch : 9 	 Loss : 0.000005
Train Epoch : 